In [2]:
from jsonschema import validate
import json

def validar_datos(schema_file, data_file):
    # Cargar el esquema
    try:
        with open(schema_file, 'r') as schema_file:
            schema = json.load(schema_file)
    except Exception as e:
        print("Error al cargar el esquema:", e)
        raise
    # Cargar los datos
    try:
        with open(data_file, 'r') as data_file:
            data = json.load(data_file)
    except Exception as e:
        print("Error al cargar los datos:", e)
        raise

    # Validar los datos contra el esquema
    try:
        validate(instance=data, schema=schema)
        print("Los datos son válidos según el esquema.")
    except Exception as e:
        print("Error de validación:", e)
    return data

schema_file = 'schema-json.json'
data_file = 'ejemplo.json'
data = validar_datos(schema_file, data_file)

Los datos son válidos según el esquema.


In [9]:
from geopy import distance
import osmnx as ox
import pandas as pd
import numpy as np
import requests

class Nodo:
    def __init__(self, nombre, latitud, longitud):
        self.nombre = nombre
        self.latitud = latitud
        self.longitud = longitud
        self.persona = 0
        #self.conexiones = {}  # Diccionario para almacenar las conexiones con otros nodos
        #Una función para añadir todas las conexiones
    
    def anadir_persona(self):
        self.persona+=1
    
    # def agregar_conexion(self, nodo_destino, distancia, medios_transporte):
    #     self.conexiones[nodo_destino] = {'distancia': distancia, 'medios_transporte': medios_transporte}

    def __str__(self):
        return f"Nodo: {self.nombre} - Latitud: {self.latitud} - Longitud: {self.longitud}"

class Aeropuerto(Nodo):
    def __init__(self, nombre, latitud, longitud, iata):
        super().__init__(nombre, latitud, longitud)
        self.iata = iata
        
class Estacion(Nodo):
    def __init__(self, nombre, latitud, longitud):
        super().__init__(nombre, latitud, longitud)
    
    
class Conexion:
    def __init__(self, origen, destino):
        self.origen = origen
        self.destino = destino
        self.distancia = self.calcular_distancia()
        self.medios_transporte = self.calcular_consumo()

    def existeCamino(self, medio):
        if medio == 'avion':
            if isinstance(self.origen,Aeropuerto) and isinstance(self.destino,Aeropuerto):
                API_KEY = "CrSmr145BGGAxnDfiBsW1hoJiTY5"
                # Define the endpoint you want to access
                endpoint = "https://test.api.amadeus.com/v2/shopping/flight-offers"

                params={
                    'originLocationCode': f'{self.origen.iata}',
                    'destinationLocationCode':f'{self.destino.iata}',
                    'departureDate':'2024-02-19',
                    'adults':'1',
                    'nonStop':'true',
                    'max':'1'
                }

                # Set up the request headers with your API credentials
                headers = {
                    "Authorization": f"Bearer {API_KEY}",
                    "Content-Type": "application/json"
                }

                # Make the GET request to the Amadeus API
                response = requests.get(endpoint, headers=headers,params=params)

                # Check if the request was successful
                if response.status_code == 200:
                    # Extract and print the response data
                    data = response.json()
                    if data['meta']['count']!=0:
                        return True
                    else:
                        return False
                else:
                    # Print the error message if the request was not successful
                    return False
            else:
                return False
        
    def calcular_distancia(self):
        origenData = (self.origen.latitud, self.origen.longitud)
        destinoData = (self.destino.latitud, self.destino.longitud)
        distancia = [-1,-1,-1,-1,-1]        
        
        osrm_url = "http://router.project-osrm.org/route/v1/driving/{},{};{},{}?steps=true"

        # Format the URL with the coordinates of the starting and ending points
        url = osrm_url.format(origenData[1], origenData[0], destinoData[1], destinoData[0])

        # Send a GET request to the OSRM server
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Extract the route information from the response
            route_data = response.json()
            
            # Extract the distance and duration of the route
            ruta = route_data["routes"][0]["distance"] / 1000  # Convert to kilometers
            
            if ruta <2:
                distancia[0]=ruta       #Permitimos andar
            else:
                distancia[0]=-1
            distancia[1]=ruta           #Añadimos distancia en coche
            distancia[2]=ruta    
            if isinstance(self.origen,Estacion) and isinstance(self.destino,Estacion):
                distancia[3]=ruta*1.5   #Handicap por tren
            else:
                distancia[4]=-1
            
        else:
            print("Error:", response.status_code)
        
        ruta = distance.distance(origenData, destinoData).kilometers
        
        
            
        if self.existeCamino('avion'):
            distancia[4]=ruta
        else:
            distancia[4]=-1
            
        return distancia

    def calcular_consumo(self):
        d1 = {
            "walk": -1 if self.distancia[0] == -1 else 0,
            "coche": -1 if self.distancia[1] == -1 else 156 * self.distancia[1],
            "bus": -1 if self.distancia[2] == -1 else 68 * self.distancia[2],
            "tren": -1 if self.distancia[2] == -1 else 14 * self.distancia[2],
            "avion": -1 if self.distancia[3] == -1 else 285 * self.distancia[3]
        }
        return d1
                  

class Grafo:
    def __init__(self):
        self.nodos = {}
        self.conexiones ={}        

    def nuevo_persona(self,nodo):
        nodo.anadir_persona()
        self.nodos[nodo.nombre] = nodo
        if not isinstance(nodo,Aeropuerto):
            self.anadir_aeropuertos(nodo,50*1000)
        if not isinstance(nodo,Estacion):
            self.anadir_estaciones(nodo,15000)
        
    def conexiones_todas(self):
        for origen in self.nodos.values():
            for destino in self.nodos.values():
                if origen.nombre != destino.nombre:
                    self.conexiones[origen.nombre+'-'+destino.nombre]=Conexion(origen,destino)
                    #print(f'{origen.nombre+'-'+destino.nombre}: ')
                    #print(self.conexiones[origen.nombre+'-'+destino.nombre].medios_transporte)

    
    def agregar_nodo(self, nodo):
        self.nodos[nodo.nombre] = nodo

    def obtener_nodo(self, nombre):
        return self.nodos.get(nombre)

    def agregar_conexion(self, nodo):
        self.conexion[self.conexion.nombre] = nodo

    def obtener_conexion(self, nombre):
        return self.conexion.get(nombre)
    
    def obtener_conexiones_origen(self,nodo):
        data =[]
        for conex in self.conexiones.values():
            if conex.origen.nombre == nodo.nombre:
                data.append(conex)
        return data

    def __str__(self):
        return f"Grafo con {len(self.nodos)} nodos"

In [10]:
grafo = Grafo()

# Crear nodos
for nodo_data in data['nodos']:
    nombre = nodo_data['nombre']
    latitud = nodo_data['latitud']
    longitud = nodo_data['longitud']
    persona = nodo_data['persona']

    # Crear instancia de nodo dependiendo del tipo
    if persona == 1:
        nodo = Nodo(nombre, latitud, longitud)
    else:
        nodo = Estacion(nombre, latitud, longitud)

    # Agregar nodo al grafo
    grafo.agregar_nodo(nodo)

# Crear conexiones
for conexion_data in data['conexiones']:
    nodo_origen_nombre = conexion_data['nodo_origen']
    nodo_destino_nombre = conexion_data['nodo_destino']
    medios_transporte = conexion_data['medios_transporte']

    nodo_origen = grafo.obtener_nodo(nodo_origen_nombre)
    nodo_destino = grafo.obtener_nodo(nodo_destino_nombre)

    conexion = Conexion(nodo_origen, nodo_destino)
    conexion.medios_transporte = medios_transporte

    # Agregar conexion al grafo
    grafo.conexiones[conexion.origen.nombre + '-' + conexion.destino.nombre] = conexion


In [15]:
from sklearn.cluster import KMeans
import numpy as np

# Obtener las coordenadas de los nodos
coordenadas = np.array([(nodo.latitud, nodo.longitud) for nodo in grafo.nodos.values()])

# Definir el número de clústeres (k)
k = 3

# Aplicar k-medias
kmeans = KMeans(n_clusters=k, random_state=0).fit(coordenadas)

# Obtener las etiquetas de clúster para cada nodo
etiquetas = kmeans.labels_

# Crear un diccionario para mapear nodos a sus respectivos clústeres
nodos_por_cluster = {i: [] for i in range(k)}
for i, etiqueta in enumerate(etiquetas):
    nodo = list(grafo.nodos.values())[i]
    nodos_por_cluster[etiqueta].append(nodo)

# Crear una lista para almacenar los centroides de cada clúster
centroides = kmeans.cluster_centers_

# Encontrar el índice del nodo objetivo más cercano a los centroides
objetivo = (grafo.obtener_nodo("Aeropuerto Santiago").latitud, grafo.obtener_nodo("Aeropuerto Santiago").longitud)  # Definir la posición objetivo
indice_objetivo_cercano = np.argmin([distance.distance(objetivo, centroide).kilometers for centroide in centroides])

personas = [[] for _ in range(k)] #lista de posiciones vacias

# Imprimir los nodos en cada clúster
for i in range(k):
    print(f"Clúster {i}:")
    for nodo in nodos_por_cluster[i]:
        print(nodo.nombre)
        if nodo.persona > 0:
            distancia_a_objetivo = distance.distance((nodo.latitud, nodo.longitud), objetivo).kilometers
            personas[i].append((nodo, distancia_a_objetivo))
        print(personas[i])
    print()
        

Clúster 0:
Tren O Burgo
[]
Bus CoruÃ±a
[]
Tren CoruÃ±a
[]
Aeropuerto CoruÃ±a
[]

Clúster 1:
Aeropuerto Santiago
[]
Tren Santiago
[]

Clúster 2:
InÃ©s
[]
Rober
[]



c:\Users\iquir\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [14]:
from geopy import distance

# Para cada clúster
for i in range(k):
    print(f"Clúster {i}:")
    
    # Verificar si hay personas en este clúster
    if personas[i]:
        # Lista para almacenar las posiciones de las personas en este clúster
        posiciones_personas = [posicion for nodo, posicion in personas[i]]

        # Calcular el punto medio de todas las posiciones de las personas en este clúster
        punto_medio = (sum(latitud for latitud, longitud in posiciones_personas) / len(posiciones_personas),
                       sum(longitud for latitud, longitud in posiciones_personas) / len(posiciones_personas))

        # Calcular la distancia del punto medio al objetivo
        distancia_al_objetivo = distance.distance(punto_medio, objetivo).kilometers

        # Imprimir el punto medio y la distancia al objetivo
        print(f"Punto medio: {punto_medio}, Distancia al objetivo: {distancia_al_objetivo} km")
    else:
        print("No hay personas en este clúster")

Clúster 0:
No hay personas en este clúster
Clúster 1:
No hay personas en este clúster
Clúster 2:
No hay personas en este clúster


In [26]:
from jsonschema import validate
import json
import heapq

def validar_datos(schema_file, data_file):
    # Cargamos el esquema
    try:
        with open(schema_file, 'r') as schema_file:
            schema = json.load(schema_file)
    except Exception as e:
        print("Error al cargar el esquema:", e)
        raise
    
    # Cargamos los datos
    try:
        with open(data_file, 'r') as data_file:
            data = json.load(data_file)
    except Exception as e:
        print("Error al cargar los datos:", e)
        raise
    
    # Validamos los datos contra el esquema
    try:
        validate(instance=data, schema=schema)
        print("Los datos son válidos según el esquema.")
    except Exception as e:
        print("Error de validación:", e)
    return data

def dijkstra(nodos, conexiones, start):
    dijkstra_graph = {}
    for conexion in conexiones:
        orig_nodo = conexion['nodo_origen']
        dest_nodo = conexion['nodo_destino']
        emision = min(filter(lambda x: x != -1, conexion['medios_transporte']))
        medios_transporte = conexion['medios_transporte']

        if orig_nodo not in dijkstra_graph:
            dijkstra_graph[orig_nodo] = {}
        if dest_nodo not in dijkstra_graph:
            dijkstra_graph[dest_nodo] = {}
        dijkstra_graph[orig_nodo][dest_nodo] = emision
        dijkstra_graph[dest_nodo][orig_nodo] = emision
    
    heap = [(0, start)]
    emisiones = {start: (0, [start])}  # Mantenemos una lista de nodos visitados hasta ahora
    while heap:
        (emision, node) = heapq.heappop(heap)
        if node in dijkstra_graph:
            for vecino, (peso, medios_transporte) in dijkstra_graph[node].items():
                nueva_emision = emision + peso
                if vecino not in emisiones or nueva_emision < emisiones[vecino][0]:
                    nuevo_camino = emisiones[node][1] + [vecino]  # Nuevo camino
                    emisiones[vecino] = (nueva_emision, nuevo_camino)
                    heapq.heappush(heap, (nueva_emision, vecino))
    for nodo in emisiones:
        print(nodo)
    return emisiones

def funcion_opt(nodos, conexiones, start_node, end_node):
    emisiones_minimas = dijkstra(nodos, conexiones, start_node)
    print(emisiones_minimas[end_node])
    emision_minima, shortest_path = emisiones_minimas[end_node]
    
    # Obtener los medios de transporte utilizados en cada conexión del camino óptimo
    medios_transporte_camino = []
    for i in range(len(shortest_path) - 1):
        origen = shortest_path[i]
        destino = shortest_path[i+1]
        medio_transporte = None
        for conexion in conexiones:
            if (conexion['nodo_origen'] == origen and conexion['nodo_destino'] == destino) or \
               (conexion['nodo_origen'] == destino and conexion['nodo_destino'] == origen):
                # Obtener el índice del medio de transporte utilizado
                medio_transporte = conexion['medios_transporte'].index(min(filter(lambda x: x != -1, conexion['medios_transporte'])))
                medios_transporte_camino.append((origen, destino, medio_transporte))
                
                # Modificar el costo si dos personas comparten un coche
                #if medio_transporte == 1:  # Si el medio de transporte es el coche
                    # Verificar si hay más de una persona en el nodo
                #    num_personas_nodo_origen = next((nodo["persona"] for nodo in nodos if nodo["nombre"] == origen), None)
                #    num_personas_nodo_destino = next((nodo["persona"] for nodo in nodos if nodo["nombre"] == destino), None)
                #    if num_personas_nodo_origen == 2 and num_personas_nodo_destino == 2:
                        # Si dos personas comparten un coche, dividir el costo por la mitad
                #        conexion['medios_transporte'][1] /= 2
                #break
    print("Camino óptimo:", shortest_path)  # Imprimir el camino óptimo
    return emision_minima, medios_transporte_camino


schema = 'schema-json.json'
data_file = 'ejemplo.json'
data = validar_datos(schema, data_file)

start_node = "Inés"
end_node = "Aeropuerto Santiago"
co2_optimized_distance, medios_transporte_camino = funcion_opt(data["nodos"], data["conexiones"], start_node, end_node)
print("Distancia óptima para Inés:", co2_optimized_distance)
for origen, destino, medios_transporte in medios_transporte_camino:
    if medios_transporte == 0:
        print(f"De {origen} a {destino}: andando")
    elif medios_transporte == 1:
        print(f"De {origen} a {destino}: coche")
    elif medios_transporte == 2:
        print(f"De {origen} a {destino}: tren")
    elif medios_transporte == 3:
        print(f"De {origen} a {destino}: avión")


start_node = "Rober"
co2_optimized_distance, medios_transporte_camino = funcion_opt(data["nodos"], data["conexiones"], start_node, end_node)
print("Distancia óptima para Rober:", co2_optimized_distance)
for origen, destino, medios_transporte in medios_transporte_camino:
    print(f"De {origen} a {destino}: {medios_transporte}")



Los datos son válidos según el esquema.
Inés


KeyError: 'Aeropuerto Santiago'

In [28]:
from jsonschema import validate
import json
import heapq

def validar_datos(schema_file, data_file):
    # Cargamos el esquema
    try:
        with open(schema_file, 'r') as schema_file:
            schema = json.load(schema_file)
    except Exception as e:
        print("Error al cargar el esquema:", e)
        raise
    
    # Cargamos los datos
    try:
        with open(data_file, 'r') as data_file:
            data = json.load(data_file)
    except Exception as e:
        print("Error al cargar los datos:", e)
        raise
    
    # Validamos los datos contra el esquema
    try:
        validate(instance=data, schema=schema)
        print("Los datos son válidos según el esquema.")
    except Exception as e:
        print("Error de validación:", e)
    return data

def dijkstra(nodos, conexiones, start):
    dijkstra_graph = {}
    for conexion in conexiones:
        orig_node = conexion['nodo_origen']
        dest_node = conexion['nodo_destino']
        distance = min(filter(lambda x: x != -1, conexion['medios_transporte']))
        medios_transporte = conexion['medios_transporte']  # Guardar los medios de transporte utilizados
        
        if orig_node not in dijkstra_graph:
            dijkstra_graph[orig_node] = {}
        if dest_node not in dijkstra_graph:
            dijkstra_graph[dest_node] = {}
        dijkstra_graph[orig_node][dest_node] = (distance, medios_transporte)  # Guardar la distancia y los medios de transporte
        dijkstra_graph[dest_node][orig_node] = (distance, medios_transporte)
    
    heap = [(0, start)]
    distances = {start: (0, [start])}  # Mantenemos una lista de nodos visitados hasta ahora
    while heap:
        (distance, node) = heapq.heappop(heap)
        #
        #modificar_persona_nodo(nodos, node, 0)
        #
        if node in dijkstra_graph:
            for neighbor, (weight, medios_transporte) in dijkstra_graph[node].items():
                new_distance = distance + weight
                if neighbor not in distances or new_distance < distances[neighbor][0]:
                    new_path = distances[node][1] + [neighbor]  # Nuevo camino
                    distances[neighbor] = (new_distance, new_path)
                    heapq.heappush(heap, (new_distance, neighbor))
    return distances

def optimize_travel(nodos, conexiones, start_node, end_node):
    shortest_distances = dijkstra(nodos, conexiones, start_node)
    shortest_distance, shortest_path = shortest_distances[end_node]
    
    # Imprimir el camino óptimo
    print("Camino óptimo:", shortest_path)
    
    # Obtener los medios de transporte utilizados en cada conexión del camino óptimo
    medios_transporte_camino = []
    for i in range(len(shortest_path) - 1):
        origen = shortest_path[i]
        destino = shortest_path[i+1]
        medio_transporte = None
        for conexion in conexiones:
            if (conexion['nodo_origen'] == origen and conexion['nodo_destino'] == destino) or \
               (conexion['nodo_origen'] == destino and conexion['nodo_destino'] == origen):
                # Obtener el índice del medio de transporte utilizado
                medio_transporte = conexion['medios_transporte'].index(min(filter(lambda x: x != -1, conexion['medios_transporte'])))
                medios_transporte_camino.append((origen, destino, medio_transporte))
                
                # Modificar el costo si dos personas comparten un coche
                if medio_transporte == 1:  # Si el medio de transporte es el coche
                    # Verificar si hay más de una persona en el nodo
                    num_personas_nodo_origen = next((nodo["persona"] for nodo in nodos if nodo["nombre"] == origen), None)
                    num_personas_nodo_destino = next((nodo["persona"] for nodo in nodos if nodo["nombre"] == destino), None)
                    if num_personas_nodo_origen == 2 and num_personas_nodo_destino == 2:
                        # Si dos personas comparten un coche, dividir el costo por la mitad
                        conexion['medios_transporte'][1] /= 2
                break
    
    return shortest_distance, medios_transporte_camino

schema = 'schema-json.json'
data_file = 'ejemplo.json'
data = validar_datos(schema, data_file)



Los datos son válidos según el esquema.


In [29]:
# Ejemplo de uso
start_node = "Inés"
end_node = "Aeropuerto Santiago"
co2_optimized_distance, medios_transporte_camino = optimize_travel(data["nodos"], data["conexiones"], start_node, end_node)
print("Distancia óptima para Inés:", co2_optimized_distance)
print("Medios de transporte utilizados en el camino óptimo:")
for origen, destino, medios_transporte in medios_transporte_camino:
    print(f"De {origen} a {destino}: {medios_transporte}")
    
start_node = "Rober"
co2_optimized_distance = optimize_travel(data["nodos"], data["conexiones"], start_node, end_node)
print("Distancia óptima para Rober:", co2_optimized_distance)
print("Medios de transporte utilizados en el camino óptimo:")
for origen, destino, medios_transporte in medios_transporte_camino:
    print(f"De {origen} a {destino}: {medios_transporte}")

KeyError: 'Aeropuerto Santiago'

In [32]:
from jsonschema import validate
import json

def validar_datos(schema_file, data_file):
    # Cargar el esquema
    try:
        with open(schema_file, 'r') as schema_file:
            schema = json.load(schema_file)
    except Exception as e:
        print("Error al cargar el esquema:", e)
        raise
    # Cargar los datos
    try:
        with open(data_file, 'r') as data_file:
            data = json.load(data_file)
    except Exception as e:
        print("Error al cargar los datos:", e)
        raise

    # Validar los datos contra el esquema
    try:
        validate(instance=data, schema=schema)
        print("Los datos son válidos según el esquema.")
    except Exception as e:
        print("Error de validación:", e)
    return data

schema_file = 'schema-json.json'
data_file = 'ejemplo.json'
data = validar_datos(schema_file, data_file)

Los datos son válidos según el esquema.


In [33]:
import heapq

def dijkstra(nodos, conexiones, start):
    dijkstra_graph = {}
    for conexion in conexiones:
        orig_node = conexion['nodo_origen']
        dest_node = conexion['nodo_destino']
        distance = min(filter(lambda x: x != -1, conexion['medios_transporte']))
        if orig_node not in dijkstra_graph:
            dijkstra_graph[orig_node] = {}
        if dest_node not in dijkstra_graph:
            dijkstra_graph[dest_node] = {}
        dijkstra_graph[orig_node][dest_node] = distance
        dijkstra_graph[dest_node][orig_node] = distance
    
    heap = [(0, start)]
    distances = {start: (0, [start])}  # Mantenemos una lista de nodos visitados hasta ahora
    while heap:
        (distance, node) = heapq.heappop(heap)
        if node in dijkstra_graph:
            for neighbor, weight in dijkstra_graph[node].items():
                new_distance = distance + weight
                if neighbor not in distances or new_distance < distances[neighbor][0]:
                    new_path = distances[node][1] + [neighbor]  # Nuevo camino
                    distances[neighbor] = (new_distance, new_path)
                    heapq.heappush(heap, (new_distance, neighbor))
    return distances

def optimize_travel(nodos, conexiones, start_node, end_node, group_travel=False):
    shortest_distances = dijkstra(nodos, conexiones, start_node)
    shortest_distance, shortest_path = shortest_distances[end_node]
    
    if group_travel:
        shortest_distance = shortest_distance / 2
    
    print("Camino óptimo:", shortest_path)  # Imprimir el camino óptimo
    return shortest_distance


start_node = "Inés"
end_node = "Aeropuerto Santiago"
co2_optimized_distance = optimize_travel(data["nodos"], data["conexiones"], start_node, end_node)
print("Distancia óptima para Inés:", co2_optimized_distance)

start_node = "Rober"
co2_optimized_distance = optimize_travel(data["nodos"], data["conexiones"], start_node, end_node)
print("Distancia óptima para Rober:", co2_optimized_distance)

KeyError: 'Aeropuerto Santiago'